In [46]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json
import re
import pickle

In [6]:
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

# Establish cursor
cursor = cnx.cursor()

In [ ]:
## function to close the connection
def close_connections():
    cursor.close()
    conn.close()

In [7]:
#name the db
db_name = 'Mod_1_Project'

In [8]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [54]:
#get list of movies from box office mojo data
titles = [x['title'] for x in data]
titles

["Tyler Perry's Boo 2! A Madea Halloween",
 "Daddy's Home 2",
 'Everything, Everything',
 'Monster Trucks',
 'How to be a Latin Lover',
 'The Nut Job 2:Nutty by Nature',
 'American Assassin',
 'Annabelle:Creation',
 'Star Wars:The Last Jedi',
 'Snatched',
 'Baby Driver',
 'Resident Evil:The Final Chapter',
 'Get Out',
 'Ferdinand',
 'Kong:Skull Island',
 'Pitch Perfect 3',
 'Rough Night',
 'Transformers:The Last Knight',
 'Fist Fight',
 'The Dark Tower',
 'American Made',
 'Underworld:Blood Wars',
 'Darkest Hour',
 'The Foreigner',
 'Home Again',
 "A Dog's Purpose",
 "The Hitman's Bodyguard",
 'Justice League',
 'Life (2017)',
 'Rings',
 'The Fate of the Furious',
 'The Mountain Between Us',
 'Murder on the Orient Express (2017)',
 'King Arthur:Legend of the Sword',
 'Smurfs:The Lost Village',
 'Geostorm',
 'Dunkirk',
 'Pirates of the Caribbean:Dead Men Tell No Tales',
 'Valerian and the City of a Thousand Planets',
 'Going in Style (2017)',
 'Power Rangers (2017)',
 'Alien:Covenant',


In [75]:

# (set(titles[0:200])-set([x['Title'] for x in new_dict if 'Title' in x.keys()]))-(set([x['Title'] for x in new_dict if 'Title' in x.keys()])-set(titles[0:200]))
# new_dict

{'A Question Of Faith',
 'Alien:Covenant',
 'Along with the Gods:The Two Worlds',
 'An Inconvenient Sequel:Truth to Power',
 'Annabelle:Creation',
 'Baahubali 2:The Conclusion',
 'Ballerina (2017) [Canada Only]',
 'Beatriz At Dinner',
 'Beauty and the Beast (2017)',
 'CHiPs',
 'Captain Underpants:The First Epic Movie',
 'Close Encounters of the Third Kind (40th Anniversary)',
 "Daddy's Home 2",
 'Diary of a Wimpy Kid:The Long Haul',
 'Flatliners (2017)',
 'Ghost in the Shell (2017)',
 'Going in Style (2017)',
 'Hazlo Como Hombre (Do It Like An Hombre)',
 'How to be a Latin Lover',
 'I am Not Your Negro',
 'In Our Hands:The Battle for Jerusalem',
 'It Comes At Night',
 'Jeepers Creepers 3',
 'John Wick:Chapter Two',
 'Jumanji:Welcome to the Jungle',
 'Kidnap (2017)',
 'King Arthur:Legend of the Sword',
 'Kingsman:The Golden Circle',
 'Kong:Skull Island',
 'Let there be Light',
 'Life (2017)',
 'Logan (2017)',
 'Murder on the Orient Express (2017)',
 'My Little Pony:The Movie',
 'Newsies

In [59]:
#function that splits titles from BOM that come with years so that we can use the list to call OMDB w/out the title
def find_year_in_title(title):
    year = re.findall('(201\d)', title)
    if year:
        return (title.replace("("+year[0]+")", ""), year[0])
    else:
        return title

list_of_movie_titles = []
for title in titles:
    list_of_movie_titles.append(find_year_in_title(title))
    
list_of_movie_titles

["Tyler Perry's Boo 2! A Madea Halloween",
 "Daddy's Home 2",
 'Everything, Everything',
 'Monster Trucks',
 'How to be a Latin Lover',
 'The Nut Job 2:Nutty by Nature',
 'American Assassin',
 'Annabelle:Creation',
 'Star Wars:The Last Jedi',
 'Snatched',
 'Baby Driver',
 'Resident Evil:The Final Chapter',
 'Get Out',
 'Ferdinand',
 'Kong:Skull Island',
 'Pitch Perfect 3',
 'Rough Night',
 'Transformers:The Last Knight',
 'Fist Fight',
 'The Dark Tower',
 'American Made',
 'Underworld:Blood Wars',
 'Darkest Hour',
 'The Foreigner',
 'Home Again',
 "A Dog's Purpose",
 "The Hitman's Bodyguard",
 'Justice League',
 ('Life ', '2017'),
 'Rings',
 'The Fate of the Furious',
 'The Mountain Between Us',
 ('Murder on the Orient Express ', '2017'),
 'King Arthur:Legend of the Sword',
 'Smurfs:The Lost Village',
 'Geostorm',
 'Dunkirk',
 'Pirates of the Caribbean:Dead Men Tell No Tales',
 'Valerian and the City of a Thousand Planets',
 ('Going in Style ', '2017'),
 ('Power Rangers ', '2017'),
 'A

In [71]:
key = config.apiKey
url = 'http://www.omdbapi.com/?i=tt3896198&apikey='

# Make call to API
# Iterate over list of titles (from box office mojo) to retrieve all

def OMDB_call(titles):
    all_titles = []
    for title in titles:
        if type(title) == tuple:
            parameters = {'t': title[0], 'y': title[1]}
        else:
            parameters = {'t': title}
        response = requests.get(url + key, params=parameters)    
        data = response.json()    
        all_titles.append(data)
        time.sleep(.5)
        # show progress
        if titles.index(title) % 10 == 0:
            print("progress:", titles.index(title),"/", len(all_titles))
    return all_titles

OMDB_call('I Am Not Your Negro')

progress: 0 / 1


[{'Title': 'Star Wars: Episode I - The Phantom Menace',
  'Year': '1999',
  'Rated': 'PG',
  'Released': '19 May 1999',
  'Runtime': '136 min',
  'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
  'Director': 'George Lucas',
  'Writer': 'George Lucas',
  'Actors': 'Liam Neeson, Ewan McGregor, Natalie Portman, Jake Lloyd',
  'Plot': 'Two Jedi escape a hostile blockade to find allies and come across a young boy who may bring balance to the Force, but the long dormant Sith resurface to claim their old glory.',
  'Language': 'English',
  'Country': 'USA',
  'Awards': 'Nominated for 3 Oscars. Another 26 wins & 65 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BYTRhNjcwNWQtMGJmMi00NmQyLWE2YzItODVmMTdjNWI0ZDA2XkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '54%'},
   {'Source': 'Metacritic', 'Value': '51/100'}],
  'Metascore': '51',
  'imdbRating': '6.5',
  'imdbVot

In [48]:
pickle.dump(movie_dict, outfile)
outfile.close

<function BufferedWriter.close>

In [45]:
# print(movie_dict[0])
# # movie_dict[17]
# movie_dict[18]
def check_for_title(data):
    movies_dict = []
    for movie in movie_dict:
        if 'Title' in movie.keys():
            movies_dict.append(movie)
    return movies_dict
movies_dict = check_for_title(movie_dict)

['Logan (2017)',
 'Murder on the Orient Express (2017)',
 'Kidnap (2017)',
 'Ballerina (2017) [Canada Only]',
 'Transformers:The Last Knight',
 'Only The Brave',
 'Newsies:The Broadway Musical',
 'Ghost in the Shell (2017)',
 'How to be a Latin Lover',
 'Jumanji:Welcome to the Jungle',
 'mother!',
 'Baahubali 2:The Conclusion',
 'Beauty and the Beast (2017)',
 'Youth (2017)',
 'Jeepers Creepers 3',
 'Smurfs:The Lost Village',
 'Alien:Covenant',
 'Let there be Light',
 'Life (2017)',
 'xXx:The Return of Xander Cage',
 'Beatriz At Dinner',
 'My Little Pony:The Movie',
 'The LEGO Batman Movie',
 'Along with the Gods:The Two Worlds',
 'Close Encounters of the Third Kind (40th Anniversary)',
 'The LEGO Ninjago Movie',
 'Thank You for Your Service (2017)',
 'An Inconvenient Sequel:Truth to Power',
 'Annabelle:Creation',
 'In Our Hands:The Battle for Jerusalem',
 "Daddy's Home 2",
 'Pirates of the Caribbean:Dead Men Tell No Tales',
 'I am Not Your Negro',
 'Star Wars:The Last Jedi',
 'T2:Trai

In [35]:
#function to create tables in our database
def create_table(dict_of_tables, db_name):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        cursor.execute("USE {}".format(db_name))
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")


In [187]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  title text NOT NULL,"
    "  budget int(10),"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nominations int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL AUTO_INCREMENT,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL AUTO_INCREMENT,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

create_table(TABLES, 'Mod_1_Project')

Creating table main_movie_table: OK
Creating table genres: already exists.
Creating table key_words: already exists.
Creating table movies_genres: already exists.
Creating table movies_keywords: already exists.
Creating table actors: already exists.
Creating table movie_actors: already exists.


In [150]:
## Function to drop all tables to refresh
def drop_everything():
    cursor.execute("""DROP TABLE Reviews;""")
    conn.commit()
    cursor.execute("""DROP TABLE Businesses;""")
    conn.commit()


In [ ]:
#insert some of the movie data into tables
def add_movies_toDB():
    for movie in movies_dict:
        cursor.execute('''insert into main_movie_table 
        (title, release_date, RT_rating, IMDB_rating) values(%s,%s,%s,%s)''',
        movie['Title'], movie['Released'], movie['Rating'][1]['Value'].replace('%', ''), movie['imdbRating']) 
        conn.commit
    
#insert the genre data into genre table         
def add_genres_toDB():
    for movie in movies_dict:
        ('''insert into genres (genre) values(%s)''', movie['Genre'])
        conn.commit
            
